# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 8 2022 9:49AM,250270,10,8746958,"Snap Medical Industries, LLC",Released
1,Nov 8 2022 9:49AM,250270,10,8746959,"Snap Medical Industries, LLC",Released
2,Nov 8 2022 9:49AM,250270,10,8746960,"Snap Medical Industries, LLC",Released
3,Nov 8 2022 9:49AM,250270,10,8746961,"Snap Medical Industries, LLC",Released
4,Nov 8 2022 9:42AM,250269,10,0086150656,ISDIN Corporation,Released
5,Nov 8 2022 9:42AM,250269,10,0086150657,ISDIN Corporation,Released
6,Nov 8 2022 9:38AM,250268,10,8749598,"Goodwin Biotechnology, Inc.",Released
7,Nov 8 2022 9:28AM,250267,19,MSP215476-BO,Methapharm-G,Released
8,Nov 8 2022 9:09AM,250264,10,0086149463,ISDIN Corporation,Released
9,Nov 8 2022 9:09AM,250264,10,0086149483,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,250266,Released,1
26,250267,Released,1
27,250268,Released,1
28,250269,Released,2
29,250270,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
250266,NaN,NaN,1.0
250267,NaN,NaN,1.0
250268,NaN,NaN,1.0
250269,NaN,NaN,2.0
250270,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
250139,0.0,0.0,1.0
250142,0.0,1.0,0.0
250147,0.0,1.0,0.0
250148,0.0,0.0,1.0
250150,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
250139,0.0,0,1
250142,0.0,1,0
250147,0.0,1,0
250148,0.0,0,1
250150,0.0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,250139,0.0,0,1
1,250142,0.0,1,0
2,250147,0.0,1,0
3,250148,0.0,0,1
4,250150,0.0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,250139,0.0,,1
1,250142,0.0,1,
2,250147,0.0,1,
3,250148,0.0,,1
4,250150,0.0,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 8 2022 9:49AM,250270,10,"Snap Medical Industries, LLC"
4,Nov 8 2022 9:42AM,250269,10,ISDIN Corporation
6,Nov 8 2022 9:38AM,250268,10,"Goodwin Biotechnology, Inc."
7,Nov 8 2022 9:28AM,250267,19,Methapharm-G
8,Nov 8 2022 9:09AM,250264,10,ISDIN Corporation
17,Nov 8 2022 9:09AM,250263,10,ISDIN Corporation
80,Nov 8 2022 9:04AM,250266,10,"Virtue Rx, LLC"
81,Nov 8 2022 8:57AM,250262,12,SugarBear
82,Nov 8 2022 8:53AM,250261,10,ISDIN Corporation
93,Nov 8 2022 8:52AM,250248,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Nov 8 2022 9:49AM,250270,10,"Snap Medical Industries, LLC",0.0,,4
1,Nov 8 2022 9:42AM,250269,10,ISDIN Corporation,0.0,,2
2,Nov 8 2022 9:38AM,250268,10,"Goodwin Biotechnology, Inc.",0.0,,1
3,Nov 8 2022 9:28AM,250267,19,Methapharm-G,0.0,,1
4,Nov 8 2022 9:09AM,250264,10,ISDIN Corporation,0.0,,9
5,Nov 8 2022 9:09AM,250263,10,ISDIN Corporation,0.0,,63
6,Nov 8 2022 9:04AM,250266,10,"Virtue Rx, LLC",0.0,,1
7,Nov 8 2022 8:57AM,250262,12,SugarBear,0.0,,1
8,Nov 8 2022 8:53AM,250261,10,ISDIN Corporation,0.0,,11
9,Nov 8 2022 8:52AM,250248,10,Eywa Pharma Inc.,1.0,,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 8 2022 9:49AM,250270,10,"Snap Medical Industries, LLC",4,
1,Nov 8 2022 9:42AM,250269,10,ISDIN Corporation,2,
2,Nov 8 2022 9:38AM,250268,10,"Goodwin Biotechnology, Inc.",1,
3,Nov 8 2022 9:28AM,250267,19,Methapharm-G,1,
4,Nov 8 2022 9:09AM,250264,10,ISDIN Corporation,9,
5,Nov 8 2022 9:09AM,250263,10,ISDIN Corporation,63,
6,Nov 8 2022 9:04AM,250266,10,"Virtue Rx, LLC",1,
7,Nov 8 2022 8:57AM,250262,12,SugarBear,1,
8,Nov 8 2022 8:53AM,250261,10,ISDIN Corporation,11,
9,Nov 8 2022 8:52AM,250248,10,Eywa Pharma Inc.,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 8 2022 9:49AM,250270,10,"Snap Medical Industries, LLC",4,
1,Nov 8 2022 9:42AM,250269,10,ISDIN Corporation,2,
2,Nov 8 2022 9:38AM,250268,10,"Goodwin Biotechnology, Inc.",1,
3,Nov 8 2022 9:28AM,250267,19,Methapharm-G,1,
4,Nov 8 2022 9:09AM,250264,10,ISDIN Corporation,9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 8 2022 9:49AM,250270,10,"Snap Medical Industries, LLC",4.0,NaN
1,Nov 8 2022 9:42AM,250269,10,ISDIN Corporation,2.0,NaN
2,Nov 8 2022 9:38AM,250268,10,"Goodwin Biotechnology, Inc.",1.0,NaN
3,Nov 8 2022 9:28AM,250267,19,Methapharm-G,1.0,NaN
4,Nov 8 2022 9:09AM,250264,10,ISDIN Corporation,9.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 8 2022 9:49AM,250270,10,"Snap Medical Industries, LLC",4.0,0.0
1,Nov 8 2022 9:42AM,250269,10,ISDIN Corporation,2.0,0.0
2,Nov 8 2022 9:38AM,250268,10,"Goodwin Biotechnology, Inc.",1.0,0.0
3,Nov 8 2022 9:28AM,250267,19,Methapharm-G,1.0,0.0
4,Nov 8 2022 9:09AM,250264,10,ISDIN Corporation,9.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3003004,95.0,12.0
12,250262,1.0,0.0
15,500333,106.0,0.0
16,250244,1.0,0.0
19,1000870,3.0,11.0
20,250253,1.0,21.0
21,1751342,6.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3003004,95.0,12.0
1,12,250262,1.0,0.0
2,15,500333,106.0,0.0
3,16,250244,1.0,0.0
4,19,1000870,3.0,11.0
5,20,250253,1.0,21.0
6,21,1751342,6.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,95.0,12.0
1,12,1.0,0.0
2,15,106.0,0.0
3,16,1.0,0.0
4,19,3.0,11.0
5,20,1.0,21.0
6,21,6.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,95.0
1,12,Released,1.0
2,15,Released,106.0
3,16,Released,1.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,12.0,0.0,0.0,0.0,11.0,21.0,1.0
Released,95.0,1.0,106.0,1.0,3.0,1.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,12.0,0.0,0.0,0.0,11.0,21.0,1.0
1,Released,95.0,1.0,106.0,1.0,3.0,1.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,12.0,0.0,0.0,0.0,11.0,21.0,1.0
1,Released,95.0,1.0,106.0,1.0,3.0,1.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()